There is a group of about 300 people who want to help each other. The problem is that a single person can meaningfully keep track of 150 people but all 300 want to cooperate and do it fairly to each other. How can they do that?

It's sad that you will not find the problem in any school book. If you did, the initial answer would be simple. Why don't they write down who helped whom and how much. For convinience they even can use 1 help unit called hlp.

In [1]:
from dataclasses import dataclass, field
from time import time

@dataclass
class Transaction:
    recipient: str
    sender: str
    amount: float
    timestamp: float = field(default_factory=time)

block = []

Now we can just put them on the chain.

In [2]:
block.append(Transaction(
    sender='Alise', recipient='Bob', amount=2.0
))
block.append(Transaction(
    sender='Bob', recipient='Jon', amount=4.5
))

In [3]:
block

[Transaction(recipient='Bob', sender='Alise', amount=2.0, timestamp=1542290722.4895535),
 Transaction(recipient='Jon', sender='Bob', amount=4.5, timestamp=1542290722.4896066)]

This covers direct person to person cooperation. Everyone is happy. If someone tries to cooperate inderectly, things fall apart. Since anyone can update the chain, transactions can be removed. If people try to exchange units of help there is an insentive to mess with the transaction list. Hm, how can we fix it? Let's chain those blocks so transactions can only be added.

In [4]:
import hashlib

class Block:
    transactions: list
    previous_hash: str = 'coinbase'

    def __init__(self):
        self.transactions = []

    def append(self, t: Transaction):
        self.transactions.append(t)
    
    def __repr__(self):
        return 'transactions: {}\n\nprevious hash: {}'.format(self.transactions.__repr__(), self.previous_hash)
    
    def hash(self):
        return hashlib.sha256(str(self).encode()).hexdigest()
        

In [5]:
block = Block()
block.append(Transaction(
    sender='Alise', recipient='Bob', amount=2.0
))
block.append(Transaction(
    sender='Bob', recipient='Jon', amount=4.5
))

In [6]:
block

transactions: [Transaction(recipient='Bob', sender='Alise', amount=2.0, timestamp=1542290724.8977764), Transaction(recipient='Jon', sender='Bob', amount=4.5, timestamp=1542290724.8978179)]

previous hash: coinbase

In [7]:
block.hash()

'422c5105a2f1b9cc90d3ffe2d2393340d9ecdd1e0e216fe9401129d0b25e9988'

Hey, look at that! We got short and unique (for our purpuses) representation of our block and all the transactions in it.

In [8]:
block.hash()

'422c5105a2f1b9cc90d3ffe2d2393340d9ecdd1e0e216fe9401129d0b25e9988'

In [9]:
block.append(Transaction(
    sender='Jon', recipient='Bob', amount=1.0
))

In [10]:
block.hash()

'539797596c7c36b4b1445b5ae2e0791bcda1135a362ef6991c7092de480a2004'

We need to represent a block in something hashable. We don't need anything fancy at this point so `__repr__` will give us what we need. Now let's add smarter chain class.

In [11]:
import itertools


class Chain:
    blocks: list

    def __init__(self):
        """We need initial block"""
        block = Block()
        block.append(Transaction(
            sender='Alise', recipient='Alise', amount=2.0
        ))
        self.blocks = [block]

    def push(self, block: Block):
        block.previous_hash = self.blocks[-1].hash()
        self.blocks.append(block)

    @property
    def tempared(self):
        """
        It gives you possition of invalid block according to it's naighbour to the right.
        If chain looks good it returns 0.
        """
        a, b = itertools.tee(self.blocks)
        next(b, None)
        for position, pair in enumerate(zip(a, b)):
            if pair[0].hash() != pair[1].previous_hash:
                return position
        return -1

    @property
    def is_valid(self):
        if self.tempared < 0:
            return True
        return False
        

In [12]:
chain = Chain()

In [13]:
chain.blocks

[transactions: [Transaction(recipient='Alise', sender='Alise', amount=2.0, timestamp=1542290728.6414185)]
 
 previous hash: coinbase]

In [14]:
for _ in range(10):
    block = Block()
    for j in range(3):
        block.append(Transaction(sender='Bob', recipient='Jon', amount=j))
    chain.push(block)

In [15]:
chain.is_valid

True

In [16]:
borrowed = chain.blocks[0].transactions.pop()
borrowed

Transaction(recipient='Alise', sender='Alise', amount=2.0, timestamp=1542290728.6414185)

In [17]:
chain.is_valid

False

In [18]:
chain.tempared

0

Let's put borrowed transaction back.

In [19]:
chain.blocks[0].transactions.append(borrowed)

In [20]:
chain.is_valid

True

We can only mess with transactions in the last block.

In [21]:
chain.blocks[-1].append(Transaction(recipient='Alise', sender='Jon', amount=5))

In [22]:
chain.is_valid

True

Now who should have a right to create blocks and add them to the chain? Everyone who wants to do it! We just need to make sure it's not monopolized by a single person. Somehow we need to give equal-ish chance for everyone to verify previous blocks and add newones. Here comes the elegant part. What if we allow previous hash to only start with `00`. Our block will have a special `nonce` field that can have anything that makes our block hash to start with `00`. If we use good hashing algorithm there is no other way but to try different possibilities untill we get what we need. This kind help work should be rewarded. We'll allow that person to assign help units to herself out of nothing.

In [23]:
class Block:
    transactions: list
    previous_hash: str = 'coinbase'
    nonce: str = 'blalba'

    def __init__(self):
        self.transactions = []

    def append(self, t: Transaction):
        self.transactions.append(t)
    
    def __repr__(self):
        return 'transactions: {}\n\nprevious hash: {}\n\nnonce: {}'.format(
            self.transactions.__repr__(), self.previous_hash, self.nonce
        )
    
    def hash(self):
        return hashlib.sha256(str(self).encode()).hexdigest()
        

In [24]:
from uuid import uuid4

def add_block_with_proof_of_work(chain: Chain, block: Block):
    last_block = chain.blocks[-1]
    for i in range(100000):
        last_block.nonce = uuid4()
        temp_hash = last_block.hash()
        if temp_hash.startswith('00'):
            block.previous_hash = temp_hash
            chain.blocks.append(block)
            print('Found hash {} after {} iterations'.format(temp_hash, i))
            break
    else:
        print('After {} iterations needed hash was not found.'.format(i))    

In [25]:
chain = Chain()

In [26]:
block = Block()
for j in range(3):
    block.append(Transaction(sender='Bob', recipient='Jon', amount=j))

In [27]:
add_block_with_proof_of_work(chain, block)

Found hash 006231bc37272c451729431e823686e10c01200724b7bff33e8bc5975dfe4d82 after 39 iterations


In [28]:
chain.blocks

[transactions: [Transaction(recipient='Alise', sender='Alise', amount=2.0, timestamp=1542290733.8655944)]
 
 previous hash: coinbase
 
 nonce: e70c449b-90f7-465c-8ba8-eff8fe7a3ec4,
 transactions: [Transaction(recipient='Jon', sender='Bob', amount=0, timestamp=1542290734.2445638), Transaction(recipient='Jon', sender='Bob', amount=1, timestamp=1542290734.2445676), Transaction(recipient='Jon', sender='Bob', amount=2, timestamp=1542290734.2445695)]
 
 previous hash: 006231bc37272c451729431e823686e10c01200724b7bff33e8bc5975dfe4d82
 
 nonce: blalba]

In [29]:
chain.is_valid

True

Facinating part is that we can have any number of arbitrary rulles. As long as each helper (a.k.a miner) validates prevous work the system works. Each helper gets reward if her block becomes part of the chain and looses that reward if someone else decides it's wrong. Since no one has a guarantee to validate their own work everyone has to abide by acepted set of rules.

There is one last major problem. I can just register a transaction from Alise (since she has a lot of help units) to Bob. It will be all valid but Alise might not even know she transfared her help units to Bob.

### Asymetric encryption to the rescue

Sadly python doesn't have many encryption features in its standard library. Cryptography is a big field. There are lot's of libs that can generate keys like PyCryptodome but we need something simple here. [ecdsa](https://github.com/warner/python-ecdsa) is the simplest to use for signature purpuses. After `pip install ecdsa` we can import an use it.

In [30]:
import ecdsa
from base64 import b64decode, b64encode

sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
vk = sk.get_verifying_key()
signature = sk.sign(vk.to_string())
vk.verify(signature, vk.to_string())

True

The idea looks simple but `sk` (Signature Key) and `vk` (Verifying Key) are objects so I can't just put them into our transaction.

In [31]:
sk, vk

(<ecdsa.keys.SigningKey at 0x7f60541a7b70>,
 <ecdsa.keys.VerifyingKey at 0x7f60540fe0b8>)

There are helper methods but it still looks ugly.

In [32]:
vk.to_string()

b'[\xe7\xf4\xb8\x8cT\x91j\xba\x08y\xa4\xc2\xb8\xbd\xe1`\x1d\x884rD\x03\xc1\xb9\x94\x9b\xa4Qgm\x03bf\xba)\xc2\xf7\x0ck\xaf\xd0\x02\x81\\\x10\xd8\xee]{|\xeb\xf7=\x993\x91b51\x80\xe0h\xeb'

In [33]:
signature

b'\xddq\x1e\xdc\x02\x19\xb5\x96v\xf9\x11M\x1d{\x95\xfe\x94\xd3\x10\xd7B\x9a2u\xaan\x11}\xaf\xb5Y\x01\xe1ub\xd7[\xa1z\xe1\xcf\xdc\xc9(\xe7\xb7\x19\xb0\xaa\xec\\\xef@5T\xc1\xdd\xb91fz%\n\xab'

Let's encode them.

In [34]:
from base64 import b64encode, b64decode

In [35]:
public_key = b64encode(vk.to_string())
sign = b64encode(signature)

print('public key: {} \nsignature:  {}'.format(public_key, sign))

public key: b'W+f0uIxUkWq6CHmkwri94WAdiDRyRAPBuZSbpFFnbQNiZropwvcMa6/QAoFcENjuXXt86/c9mTORYjUxgOBo6w==' 
signature:  b'3XEe3AIZtZZ2+RFNHXuV/pTTENdCmjJ1qm4Rfa+1WQHhdWLXW6F64c/cySjntxmwquxc70A1VMHduTFmeiUKqw=='


In [36]:
# Our new transaction data
@dataclass
class Transaction:
    recipient: bytes
    sender: bytes
    signature: bytes # New signature filed
    amount: float
    timestamp: float = field(default_factory=time)

In [37]:
def sign(public: bytes, private: bytes) -> bytes:
    print(private)
    pk = b64decode(private)
    print(pk)
    signing_key = ecdsa.SigningKey.from_string(pk)
    signature = b64encode(signing_key.sign(public))
    return signature


def is_valid(transaction: Transaction) -> bool:
    pub_key = ecdsa.VerifyingKey.from_string(b64decode(transaction.sender))
    return pub_key.verify(b64decode(transaction.signature), transaction.sender)


class Person:
    public: bytes
    private: bytes

    def __init__(self):
        sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
        vk = sk.get_verifying_key()
        self.public = b64encode(vk.to_string())
        self.private = b64encode(sk.to_string())

    def __repr__(self):
        return str(self.public)

    def send_help(self, to:bytes, amount: float) -> Transaction:
        return Transaction(
            recipient=to,
            sender=self.public,
            signature=sign(self.public, self.private),
            amount=amount
        )

Now if Alis wants to send help to Bob, first she and Bob have to genarate key pairs.

In [38]:
alise = Person()
bob = Person()
jon = Person()

In [39]:
alise.send_help(to=bob.public, amount=2.0)

b'sdA2fb9OG9RQH3SWLeELJQwAtvdFGsavEZj6K1dXhdc='
b'\xb1\xd06}\xbfN\x1b\xd4P\x1ft\x96-\xe1\x0b%\x0c\x00\xb6\xf7E\x1a\xc6\xaf\x11\x98\xfa+WW\x85\xd7'


AssertionError: (32, 24)

In [40]:
ecdsa.SigningKey.from_string(b'\x13\xdf\x1c\xd0.U9\x01|\x14-.\x9a\xf7\xf9\xac\xed\x05\xa6"\x0e\x9cj*L/\xd8\xbe\x87\x08\xd0w')

AssertionError: (32, 24)